In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
# Import API key
from config1 import g_key
# Configure gmaps
gmaps.configure(api_key=g_key)







In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

City_ID        City Country      Lat       Lng  Max Temp  Humidity  \
0        0    Mingaora      PK  34.7758   72.3625     73.96        88   
1        1      Atuona      PF  -9.8000 -139.0333     76.64        72   
2        2     Mataura      NZ -46.1927  168.8643     39.47        98   
3        3  Nikolskoye      RU  59.7035   30.7861     65.39        67   
4        4      Maceio      BR  -9.6658  -35.7353     76.44        83   

   Cloudiness  Wind Speed Current Description  
0          27        3.85    scattered clouds  
1          40       14.23    scattered clouds  
2          31        3.60    scattered clouds  
3          53       11.39       broken clouds  
4          90        9.22     overcast clouds

In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria..
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 80
What is the maximum temperature you would like for your vacation? 90


In [5]:
# 3. Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

City_ID         City Country      Lat       Lng  Max Temp  Humidity  \
6         6        Kapaa      US  22.0752 -159.3190     80.92        80   
7         7    Hithadhoo      MV  -0.6000   73.0833     80.78        73   
16       16     Gonaives      HT  19.4500  -72.6833     87.78        51   
19       19     Katsuura      JP  35.1333  140.3000     80.71        88   
26       26  Yellowknife      CA  62.4560 -114.3525     83.32        36   
27       27      Cayenne      GF   4.9333  -52.3333     87.84        76   
39       39     Victoria      HK  22.2855  114.1577     85.32        88   
52       52  Bodden Town      KY  19.2833  -81.2500     90.00        66   
60       60        Dakar      SN  14.6937  -17.4441     87.93        66   
61       61          Gat      IL  31.6100   34.7642     87.40        53   

    Cloudiness  Wind Speed Current Description  
6           90        5.99          light rain  
7          100        4.85     overcast clouds  
16          98        7.40     overcast clouds  
19          21        7.54          few clouds  
26          20        3.00          few clouds  
27          40        4.00    scattered clouds  
39         100        1.01          light rain  
52          20       11.50          few clouds  
60          20        4.61          few clouds  
61           0        5.28           clear sky

In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                123
City                   123
Country                123
Lat                    123
Lng                    123
Max Temp               123
Humidity               123
Cloudiness             123
Wind Speed             123
Current Description    123
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()
#clean_travel_cities.head()
clean_travel_cities.count()
#clean_travel_cities.columns

City_ID                123
City                   123
Country                123
Lat                    123
Lng                    123
Max Temp               123
Humidity               123
Cloudiness             123
Wind Speed             123
Current Description    123
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
# 5b. Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""
hotel_df.head(10)
hotel_df.count()


City                   123
Country                123
Max Temp               123
Current Description    123
Lat                    123
Lng                    123
Hotel Name             123
dtype: int64

In [ ]:
#when initially running this code, I was getting errors and the run would get stuck. So I decided to decrease
#the number of runs, using temperature range 80-90 F(123 cities), instead of 75-90F,(245cities found)and this made the 
#below code run.
# I am not sure why this has happened but after spending many hours trying to run the code and troubleshooting, the learning
#assistants and I decided to keep the run for 80-90F as we could not understand why the run would not run for a larger
#selection.
#various effords were made to check various stages of the code using index and iloc methods, but no obvious error, just 
#the kernel got stuck and after it reached city number 22 it gave errors, so we eliminated this city, and tried to re run 
#it again, and it got stuck on city no 35 and so on. We decided that there were something odd with some cities and 
#the nearby search.
#hotel_df.loc[270]

In [9]:
#hotel_df.reset_index(inplace=True) 
#hotel_df.drop(index=72, inplace=True)
#hotel_df.drop(index=70, inplace=True)
# 6a. Set parameters to search for a hotel
import time
params = {
   "radius": 5000,
   "type": "lodging",
   "key": g_key
}
# 6b. Iterate through the hotel DataFrame
for index, row in hotel_df.iterrows():
   # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
   # 6c. Get latitude and longitude from DataFrame.
    params["location"] = f"{lat},{lng}"
   # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
   # 6e. Make request and retrieve the JSON data from the search.
    time.sleep(1)
    hotels = requests.get(base_url, params=params).json()
  
   # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        #print(index)
    except:
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.head(15)

City Country  Max Temp Current Description      Lat       Lng  \
6            Kapaa      US     80.92          light rain  22.0752 -159.3190   
7        Hithadhoo      MV     80.78     overcast clouds  -0.6000   73.0833   
16        Gonaives      HT     87.78     overcast clouds  19.4500  -72.6833   
19        Katsuura      JP     80.71          few clouds  35.1333  140.3000   
26     Yellowknife      CA     83.32          few clouds  62.4560 -114.3525   
27         Cayenne      GF     87.84    scattered clouds   4.9333  -52.3333   
39        Victoria      HK     85.32          light rain  22.2855  114.1577   
52     Bodden Town      KY     90.00          few clouds  19.2833  -81.2500   
60           Dakar      SN     87.93          few clouds  14.6937  -17.4441   
61             Gat      IL     87.40           clear sky  31.6100   34.7642   
84      Lakhisarai      IN     83.57     overcast clouds  25.1768   86.0932   
89           Praya      CV     82.94          few clouds  14.9215  -23.5087   
93         Qandala      SO     87.98    scattered clouds  11.4720   49.8728   
94       Bathsheba      BB     86.61    scattered clouds  13.2167  -59.5167   
96  Guerrero Negro      MX     82.87           clear sky  27.9769 -114.0611   

                             Hotel Name  
6   Sheraton Kauai Coconut Beach Resort  
7                     Scoop Guest House  
16                Vertières Doux Séjour  
19              Katsuura Hotel Mikazuki  
26                   The Explorer Hotel  
27                      Hôtel Le Dronmi  
39                   Mini Hotel Central  
52                         Will T Place  
60              Redface Studios Meublés  
61                             OlusHome  
84                     Hotel DRS Deluxe  
89                        LT Aparthotel  
93                                       
94                       Atlantis Hotel  
96         Casa Laguna, Bed & Breakfast

In [11]:
hotel_df.tail(50)

City Country  Max Temp Current Description      Lat  \
458               Kadoma      JP     82.38       broken clouds  34.7333   
460               Boende      CD     83.34     overcast clouds  -0.2167   
462              Gigmoto      PH     81.16     overcast clouds  13.7796   
469             Pochutla      MX     87.62       broken clouds  15.7432   
479                 Aksu      CN     80.83     overcast clouds  41.1231   
480                  Eyl      SO     86.36     overcast clouds   7.9803   
489               Touros      BR     86.90    scattered clouds  -5.1989   
490                Goure      NE     88.02     overcast clouds  13.9835   
506               Fuzhou      CN     80.49     overcast clouds  26.0614   
509             Beatrice      US     81.86                haze  40.2681   
511               Kijang      KR     82.13     overcast clouds  35.2442   
512                Balud      PH     82.36     overcast clouds  10.1476   
520               Latung      PH     83.89    scattered clouds   5.5000   
523              Ottumwa      US     81.28                haze  41.0200   
533               Weinan      CN     82.09           clear sky  34.5036   
544        Marsh Harbour      BS     85.28           clear sky  26.5412   
546            Porbandar      IN     81.91     overcast clouds  21.6422   
547              Itarema      BR     85.66       broken clouds  -2.9248   
551          Salinopolis      BR     83.34    scattered clouds  -0.6136   
559                 Emba      KZ     84.42          few clouds  48.8264   
565              Iquitos      PE     82.44       broken clouds  -3.7481   
569              Wanning      CN     80.91     overcast clouds  18.8003   
577             Umm Lajj      SA     89.33           clear sky  25.0213   
580              Mindelo      CV     81.95    scattered clouds  16.8901   
584            Fonte Boa      BR     82.49    scattered clouds  -2.4959   
588              Wenling      CN     80.91     overcast clouds  28.3669   
597        Novyye Burasy      RU     80.94       broken clouds  52.1324   
598             Progreso      MX     89.71       broken clouds  21.2833   
599             Buchanan      LR     80.56     overcast clouds   5.8808   
600  Santa Maria Del Oro      MX     80.98    scattered clouds  25.9333   
606        San Cristobal      VE     82.58       broken clouds   7.7669   
607             Vallenar      CL     85.82           clear sky -28.5708   
612               Malard      IR     89.78    scattered clouds  35.6659   
613           San Lazaro      BZ     89.31     overcast clouds  18.0387   
620         Capitao Poco      BR     86.81    scattered clouds  -1.7464   
626                Kuryk      KZ     86.31           clear sky  43.2000   
627               Acarau      BR     87.31       broken clouds  -2.8856   
634             Lorengau      PG     81.64          few clouds  -2.0226   
639            Kavaratti      IN     82.38     overcast clouds  10.5669   
643       Puerto Penasco      MX     88.92     overcast clouds  31.3167   
647            Vinukonda      IN     87.10    scattered clouds  16.0500   
653               Davila      PH     80.76     overcast clouds  18.4726   
655               Nicoya      CR     89.15    scattered clouds  10.1483   
657               Cortes      ES     80.60       broken clouds  41.9395   
663             Hasilpur      PK     89.13     overcast clouds  29.7122   
667          Greeneville      US     82.22     overcast clouds  36.1632   
668              Omurtag      BG     80.55           clear sky  43.1000   
671              Nipawin      CA     80.64           clear sky  53.3668   
682               Pandan      PH     82.72     overcast clouds  14.0453   
689             Iracoubo      GF     82.24          few clouds   5.4802   

          Lng                                         Hotel Name  
458  135.5833                        Hotel Agora Osaka Moriguchi  
460   20.8667                                               

In [ ]:
hotel_df.loc[:,'Hotel Name']

In [12]:
# 7. Drop the rows where there is no Hotel Name.
#dropna() method
import numpy as np
import pandas as pd






In [16]:
filter=hotel_df['Hotel Name']!=""
clean_hotel_df=hotel_df[filter]
clean_hotel_df.head()

City Country  Max Temp Current Description      Lat       Lng  \
6         Kapaa      US     80.92          light rain  22.0752 -159.3190   
7     Hithadhoo      MV     80.78     overcast clouds  -0.6000   73.0833   
16     Gonaives      HT     87.78     overcast clouds  19.4500  -72.6833   
19     Katsuura      JP     80.71          few clouds  35.1333  140.3000   
26  Yellowknife      CA     83.32          few clouds  62.4560 -114.3525   

                             Hotel Name  
6   Sheraton Kauai Coconut Beach Resort  
7                     Scoop Guest House  
16                Vertières Doux Séjour  
19              Katsuura Hotel Mikazuki  
26                   The Explorer Hotel

In [32]:
# 8a. Create the output File (CSV)
output_data_file='CSV_Files/WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure

In [30]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.05)
fig.add_layer(marker_layer)

In [31]:
fig

Figure(layout=FigureLayout(height='420px'))